In [25]:
# importing required libraries

import random as rand
import pygame

In [26]:
# Union Find data structure

class DSU:
    def __init__(self, sze):
        self.par = list(range(sze))
        self.rank = [1] * sze
        self.size = sze

    def find(self, vertex):
        root = vertex
        while self.par[root] != root:
            root = self.par[root]

        while vertex != root:
            temp = self.par[vertex]
            self.par[vertex] = root
            vertex = temp

        return root

    def union_(self, vertex1, vertex2):
        root1 = self.find(vertex1)
        root2 = self.find(vertex2)

        if self.rank[root1] > self.rank[root2]:
            self.par[root2] = root1
            self.rank[root1] += self.rank[root2]
        else:
            self.par[root1] = root2
            self.rank[root2] += self.rank[root1]

    def is_connected(self, vertex1, vertex2):
        return self.find(vertex1) == self.find(vertex2)


In [35]:
#Co-ordinates to vertex number mapping and vice-versa

dct = {}
lst = []
k=0

print("Vertex Labels")

for j in range(50,650,30):
    for i in range(50,650,30):
        pt = (i+15,j+15)    
        dct[pt]=k
        lst.append(pt)
        print(k,end=' ')
        k+=1
    print()

print(dct)

Vertex Labels
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 
80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 
100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 
120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 
140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 
160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 
180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 
200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 
220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 
240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 
260 261 262 263 264 265 266 267 268 269 270

Description of the algorithm used to build the random maze generator

• Treat the grid's cells as vertices (labelled from 0 to n-1)
• Two vertices are connected by an edge if:
    i. They are next to each other
    ii. The adjoining wall is absent
• Continue adding the edges, i.e. removing walls, till vertex 0 (entry point) is connected to vertex n-1 (exit point)
• Do not add edges within the same connected components, construct a minimum spanning tree with n-1 edges

Pseudocode:

MazeGenerator(G, E):
    Input: A grid, G, consisting of n cells and a set, E, of m “walls,” each of which divides two cells, x and y, so that the walls in E initially separate and isolate all cells in G.
    Output: A subset, R of E, such that adding the edges in R to E (i.e., removing the walls present in G using the information from R) creates a maze 

    while R has less than n-1 edges do 
        Choose an edge, (x, y), in E uniformly at random    
        if find(x) ≠ find(y) then do
            union(find(x), find(y))
            Add the edge (x, y) to R

    return R

In [ ]:
# Code for Maze Generator Using PyGame

pygame.init()

# Set up display
width, height = 700,700
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Maze Generator")

# Set background color
background_color = (255, 255, 255)  # White
screen.fill(background_color)

# Draw a 20*20 grid, construct all walls (black lines)
for i in range(21):
    pygame.draw.line(screen, (0, 0, 0), (0+50, 30*i+50), (600+50, 30*i+50), 1)
    pygame.draw.line(screen, (0, 0, 0), (30*i+50, 50), (30*i+50,600+50), 1)

#Start and End points (erasing walls using white lines)
pygame.draw.line(screen, (255, 255, 255), (50,50), (50,80), 2)
pygame.draw.line(screen, (255, 255, 255), (650,620), (650,650), 2)

# Initializing DSU data structure and edge count
d = DSU(400)
edge_count= 0

# Do this until we obtain a spanning tree
while (edge_count<399):
    vert = lst[rand.randint(0,399)]    #randomly choosing a vertex
    nbr = (-1,-1)                #default neighbour

    if (vert[0]==65):                 #left vertical line
        if (vert[1]==65):               #top-left corner
            y_off=rand.randint(0,1)
        elif (vert[1]==635):             #bottom-left corner
            y_off=rand.randint(-1,0)
        else:
            y_off = rand.randint(-1,1)
        if (y_off==0):
            x_off=1
        else:
            x_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30) 

    elif (vert[0]==635):                #right vertical line
        if (vert[1]==65):               #top-right corner
            y_off=rand.randint(0,1)
        elif (vert[1]==635):             #bottom-right corner
            y_off=rand.randint(-1,0)
        else:
            y_off = rand.randint(-1,1)
        if (y_off==0):
            x_off=-1
        else:
            x_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30) 

    elif (vert[1]==65):                     #top horizontal line
        x_off = rand.randint(-1,1)           #all corner points covered, hence no need for larger if-else now onwards
        if (x_off==0):
            y_off=1
        else:
            y_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30)    

    elif (vert[1]==635):                     #bottom horizontal line
        x_off = rand.randint(-1,1)          
        if (x_off==0):
            y_off=-1
        else:
            y_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30) 

    else:                                    #all middle points
        x_off = rand.randint(-1,1)
        if (x_off==0):
            l = [-1,1]   
            y_off=rand.choice(l)   #either move-up or move-down
        else:
            y_off=0
        nbr = (vert[0]+x_off*30,vert[1]+y_off*30) 

    #code reached here, now decide whether to add edge or not

    vertex1 = dct[vert]
    vertex2 = dct[nbr]

    if (d.is_connected(vertex1,vertex2)==False):
        d.union_(vertex1,vertex2)
        edge_count+=1
        if (vert[0]==nbr[0]):
            x1 = vert[0]-15
            x2 = vert[0]+15
            y = (vert[1]+nbr[1])/2
            pygame.draw.line(screen, (255, 255, 255), (x1,y), (x2,y), 2)
        else:
            x = (vert[0]+nbr[0])/2
            y1 = vert[1]-15
            y2 = vert[1]+15
            pygame.draw.line(screen, (255, 255, 255), (x,y1), (x,y2), 2)            

pygame.display.flip()

# ----- Uncomment the following code block if you wish to save the maze in a .jpg file -----

# Capture the screen surface
screenshot = pygame.Surface((width, height))
screenshot.blit(screen, (0, 0))

# Save the surface as a .jpg file
pygame.image.save(screenshot, 'maze.jpg')

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()